### Notes in LLM Google doc

In [18]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"]='0,1'

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import textwrap
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

device = "cuda" if torch.cuda.is_available() else "cpu"
#Need only 1 GPU if loading 8-bit model
print(device)

print("Using %d GPUs" %torch.cuda.device_count())

cuda
Using 4 GPUs


In [2]:
model_name = "eachadea/vicuna-13b-1.1"
tokenizer_path = "./tokenizer/"

#Create a local tokenizer copy the first time
if os.path.isdir(tokenizer_path):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
else:
    tokenizer = AutoTokenizer.from_pretrained("model_name")
    os.mkdir(tokenizer_path)
    tokenizer.save_pretrained(tokenizer_path)


In [3]:
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")#, load_in_8bit=True)
#model = AutoModelForCausalLM.from_pretrained(model_name).to(device) -- tries to put everything on 1 GPU

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [67]:
pipe = pipeline(
    "text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=512,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)

In [68]:
#question = "How can you use coherent imaging in materials science?"
#question = "What is the capital of England?"
question = "Hi, I am Mathew"
a = pipe(question)

In [69]:
textwrap.wrap(a[0]['generated_text'])

['Hi, I am Mathew from Dubai. I have a 2016 Ford Ranger Wildtrak and I',
 'would like to know if the OEM front bumper can be replaced with an',
 'aftermarket one? Also what are the regulations in UAE regarding such',
 'modifications? Thank you!']

### See langchain prompt in action

In [70]:
local_llm = HuggingFacePipeline(pipeline=pipe)

template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 
{instruction}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["instruction"])

In [71]:
llm_chain = LLMChain(prompt=prompt, 
                     llm=local_llm
                     )

def response(question):
    return textwrap.wrap(llm_chain.run(question))

In [72]:
print (response(question))

[' Hello Mathew! How can I help you today?']


In [73]:
question2 = "What is my name?"
print (response(question2))

[' Your name is \\_\\_\\_\\_\\_\\_\\_']


### Langchain conv buffer

In [74]:
# We are going to set the memory to go back N turns
window_memory = ConversationBufferWindowMemory(k=12)

conversation = ConversationChain(
    llm=local_llm, 
    verbose=True, 
    memory=window_memory
)

In [75]:
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [80]:
c = conversation.predict(input="Isn't your name Vicuna? Wa isn't part of it. Try again")
print (c)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is your name?
AI:  My name is Vicuna, and I am a language model trained by researchers from Large Model Systems Organization (LMSYS).
Human: My name is Mathew
AI:  Nice to meet you, Mathew. How can I assist you today?
Human: Do you know what a portmanteau word is?
AI:   Yes, I do! A portmanteau word is a type of blend word that combines two or more words to create a new one with a unique meaning. For example, "smog" which is a combination of "smoke" and "fog."
Human: What is a portmanteau word from our names?
AI:   Let me think... Ah, I have it! "Mathews" could be considered a portmanteau word because it combines the first part of your name ("Matt") with th